In [7]:
import os
from datasets import load_dataset
from PIL import Image
import pandas as pd
from easyocr import Reader
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import pytesseract


In [8]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None) # Show full content of each column (no truncation)
pd.set_option('display.width', None)

In [9]:
dataset = load_dataset("ahmedheakl/arocrbench_synthesizear",split="train")


In [10]:
results = []
for sample in dataset:
    image = sample["image"]  # Already a PIL Image object
    if image.mode != "RGB":
        image = image.convert("RGB")
    ground_truth = sample["text"]
    results.append({
        "image": image,
        "ground_truth": ground_truth,
        "tesseract": None,
        "easyocr": None,
    })
print(f"Initialized results with {len(results)} samples.")

Initialized results with 500 samples.


### Tesseract Processing

In [11]:
for i, sample in enumerate(results):
    image = sample["image"]
    tesseract_text = pytesseract.image_to_string(image,lang='ara')
    results[i]['tesseract'] = tesseract_text
print("Tesseract Processing completeed")

Tesseract Processing completeed


## EasyOCR

In [12]:
easyocr_reader = Reader(['ar'], gpu=True)
for i, sample in enumerate(results):
    image = sample["image"]
    easyocr_result = easyocr_reader.readtext(image, detail=0)
    easyocr_text = " ".join(easyocr_result)
    results[i]["easyocr"] = easyocr_text
print("EasyOCR processing completed.")

EasyOCR processing completed.


In [17]:
import re

def remove_diacritics(text):
    # Define Arabic diacritics using Unicode ranges
    diacritics = [
        '\u0617', '\u0618', '\u0619', '\u061A',  # Small high signs
        '\u064B', '\u064C', '\u064D', '\u064E', '\u064F', '\u0650',  # Tanween and basic diacritics
        '\u0651', '\u0652', '\u0653', '\u0654', '\u0655', '\u0656',  # Shadda, sukun, etc.
        '\u0657', '\u0658', '\u0659', '\u065A', '\u065B', '\u065C', 
        '\u065D', '\u065E', '\u065F', '\u0670'   # Additional Arabic diacritics
    ]
    # Create a regex pattern to match any diacritic
    pattern = '[' + ''.join(diacritics) + ']'
    # Replace all diacritics with an empty string
    return re.sub(pattern, '', text)

In [20]:
df = pd.DataFrame(results)
df.head()

,image,ground_truth,tesseract,easyocr
0,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=639x114 at 0x7BF63B8D3E10>,"وَإِذا ما سَأَلَتْنِي عَن مَعْنَى لَفَظَهُ ""عرب"" عِنْدَ","وَإِذا ما سَألْْيِي عَن مَعْنَى لَفَظَهُ ""عرب"" عِنْدَ\n","لفظهً "" عرب "" عندً عن واذا ما سألتني معنى"
1,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1182x147 at 0x7BF638440450>,أَمّا فَهُم النُصُوصِ وَاِسْتِنْباط مَعانِيها بِوَجْهٍ صَحِيحٌ دقيق،,"أما فَهُم النْصْوصٍ واشتئباط قعانيها بوكو صَحيحٌ دقيق,\n",_ النضوص واستنباط قعانيها بؤجه ضجيح دقيق . أما فهم
2,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=994x163 at 0x7BF638440F10>,تُثِير فِيها الاسود، وَهُوَ ما يَتَعارَض مَعَ اِكْتِشافِ,تَثْير فيها الاسود. وَهَوٍ ما تَتَعارَض مق اكْتشافيٍ\n,تثير فيها الاسود وهو ما يتعارض مع اكتشاف
3,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1315x224 at 0x7BF638441910>,الجماعي، وَكادَت تَصِل إِلَى المَرْحَلَةِ الاِشْتِراكِيَّة لِسَيْطَرَةِ أَكْبَرَ,لَى المَرْحَلَةِ الإشترايّة لسَيْطرة أَخُبرَ\nالجماعي. وَكَادَت تصل إِلَى المَْحَلَةِ الاشترايّة لِسَيْط\n,"وكادت تصل إلى المزحلة الاشتاكيًة لسيط ة أفبر الجماعي ,"
4,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1224x129 at 0x7BF638441190>,مَعَهُ مَنْدِيلا فِيهِ جردقتان وَقَطَعَ لَحْم سَكْباج مُبَرَّد,مَعهَ منديلا فيه جردقتان وَفَطعَ لخم سه باج مبرد\n,معهً منديلا فيه جردقتان وقطع سحباج آخم مبرد


In [24]:

df['ground_truth_t'] = df["ground_truth"].apply(remove_diacritics)
df['tesseract_t'] = df["tesseract"].apply(remove_diacritics)
df['easyocr_t'] = df["easyocr"].apply(remove_diacritics)

In [25]:
df.head()


,image,ground_truth,tesseract,easyocr,ground_truth_t,tesseract_t,easyocr_t
0,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=639x114 at 0x7BF63B8D3E10>,"وَإِذا ما سَأَلَتْنِي عَن مَعْنَى لَفَظَهُ ""عرب"" عِنْدَ","وَإِذا ما سَألْْيِي عَن مَعْنَى لَفَظَهُ ""عرب"" عِنْدَ\n","لفظهً "" عرب "" عندً عن واذا ما سألتني معنى","وإذا ما سألتني عن معنى لفظه ""عرب"" عند","وإذا ما سأليي عن معنى لفظه ""عرب"" عند\n","لفظه "" عرب "" عند عن واذا ما سألتني معنى"
1,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1182x147 at 0x7BF638440450>,أَمّا فَهُم النُصُوصِ وَاِسْتِنْباط مَعانِيها بِوَجْهٍ صَحِيحٌ دقيق،,"أما فَهُم النْصْوصٍ واشتئباط قعانيها بوكو صَحيحٌ دقيق,\n",_ النضوص واستنباط قعانيها بؤجه ضجيح دقيق . أما فهم,أما فهم النصوص واستنباط معانيها بوجه صحيح دقيق،,"أما فهم النصوص واشتئباط قعانيها بوكو صحيح دقيق,\n",_ النضوص واستنباط قعانيها بؤجه ضجيح دقيق . أما فهم
2,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=994x163 at 0x7BF638440F10>,تُثِير فِيها الاسود، وَهُوَ ما يَتَعارَض مَعَ اِكْتِشافِ,تَثْير فيها الاسود. وَهَوٍ ما تَتَعارَض مق اكْتشافيٍ\n,تثير فيها الاسود وهو ما يتعارض مع اكتشاف,تثير فيها الاسود، وهو ما يتعارض مع اكتشاف,تثير فيها الاسود. وهو ما تتعارض مق اكتشافي\n,تثير فيها الاسود وهو ما يتعارض مع اكتشاف
3,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1315x224 at 0x7BF638441910>,الجماعي، وَكادَت تَصِل إِلَى المَرْحَلَةِ الاِشْتِراكِيَّة لِسَيْطَرَةِ أَكْبَرَ,لَى المَرْحَلَةِ الإشترايّة لسَيْطرة أَخُبرَ\nالجماعي. وَكَادَت تصل إِلَى المَْحَلَةِ الاشترايّة لِسَيْط\n,"وكادت تصل إلى المزحلة الاشتاكيًة لسيط ة أفبر الجماعي ,",الجماعي، وكادت تصل إلى المرحلة الاشتراكية لسيطرة أكبر,لى المرحلة الإشتراية لسيطرة أخبر\nالجماعي. وكادت تصل إلى المحلة الاشتراية لسيط\n,"وكادت تصل إلى المزحلة الاشتاكية لسيط ة أفبر الجماعي ,"
4,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1224x129 at 0x7BF638441190>,مَعَهُ مَنْدِيلا فِيهِ جردقتان وَقَطَعَ لَحْم سَكْباج مُبَرَّد,مَعهَ منديلا فيه جردقتان وَفَطعَ لخم سه باج مبرد\n,معهً منديلا فيه جردقتان وقطع سحباج آخم مبرد,معه منديلا فيه جردقتان وقطع لحم سكباج مبرد,معه منديلا فيه جردقتان وفطع لخم سه باج مبرد\n,معه منديلا فيه جردقتان وقطع سحباج آخم مبرد


In [23]:
from jiwer import wer, cer 

for model in ["tesseract","easyocr"]:
    wer_score = wer(df["ground_truth"].tolist(), df[model].tolist())
    cer_score = cer(df["ground_truth"].tolist(), df[model].tolist())
    print(f"{model} - WER: {wer_score:.2f}, CER: {cer_score:.2f}")

tesseract - WER: 1.02, CER: 0.43
easyocr - WER: 1.06, CER: 0.59


In [26]:
from jiwer import wer, cer 

for model in ["tesseract_t","easyocr_t"]:
    wer_score = wer(df["ground_truth_t"].tolist(), df[model].tolist())
    cer_score = cer(df["ground_truth_t"].tolist(), df[model].tolist())
    print(f"{model} - WER: {wer_score:.2f}, CER: {cer_score:.2f}")

tesseract_t - WER: 0.72, CER: 0.33
easyocr_t - WER: 0.76, CER: 0.46


In [27]:
df.to_csv("khattat_synthesizear_result.csv")


: 